In [ ]:
import os
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
if(os.path.exists("xtrain.pickle")):
    infile = open("xtrain.pickle", "rb")
    xtrain = pickle.load(infile)
    infile.close()

    infile = open("ytrain.pickle", "rb")
    ytrain = pickle.load(infile)
    infile.close()

    infile = open("xtest.pickle", "rb")
    xtest = pickle.load(infile)
    infile.close()

    infile = open("ytest.pickle", "rb")
    ytest = pickle.load(infile)
    infile.close()

    infile = open("xval.pickle", "rb")
    xval = pickle.load(infile)
    infile.close()

    infile = open("yval.pickle", "rb")
    yval = pickle.load(infile)
    infile.close()
else:
    print("Split dataset first!")

In [ ]:
plt.imshow(xtrain[0])
plt.show()

In [ ]:
resnet = tf.keras.applications.ResNet50(
    include_top = False, 
    weights = 'imagenet',
    input_shape = (384,512,3)
)

model = tf.keras.models.Sequential()
model.add(resnet)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.summary()

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001), 
    loss = 'sparse_categorical_crossentropy', 
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(xtrain, ytrain, 
                    batch_size=32, 
                    epochs=10, 
                    validation_data=(xval,yval))

# accuracy graph
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# loss graph
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
prediction = model.predict(xtest)

correct = 0
wrong_index = []
predicted_index = []

for i in range(len(prediction)):
    temp = np.argmax(prediction[i])
    if (temp == ytest[i]):
        correct += 1
    else:
        wrong_index.append(i)
        predicted_index.append(temp)

print("accuracy: ", correct / len(xtest) * 100)

In [ ]:
cols = 6
rows = int(len(wrong) / cols) + 1
axes = []
figs = plt.figure(figsize=(17,20))

labels = ["cardboard", "glass", "metal", 
          "paper", "plastic", "trash"]

for i in range(rows * cols):
    if(i < len(wrong)):
        axes.append(figs.add_subplot(rows, cols, i+1))
        pic = xtest[wrong_index[i]]
        name = labels[ytest[predicted_index[i]]]
        axes[-1].set_title(name)
        plt.imshow(pic)
plt.show()